# Minggu 3 — Data Cleaning & Preprocessing (Colab, v2)

Perubahan utama:
- **Imputasi eksplisit** untuk kolom:
  - `shift` (kategorik) → modus atau label `'Unknown'`
  - `machine_speed`, `temperature_c`, `downtime_min` (numerik) → median
- Dua level hasil:
  1) `df_clean_min.csv` (dedup by-features + imputasi sederhana pada kolom target)
  2) `train_ready.csv` & `test_ready.csv` (via pipeline: imputasi + One-Hot + scaling; **tanpa kebocoran**)

Catatan: tetap pertahankan **raw** apa adanya; simpan hasil ke `data/interim/` dan `data/processed/`.

In [1]:
# ⬇️ Setup dasar
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import os

pd.options.display.max_columns = 200

## 1) Load data
Opsi:
1. Upload manual ke `/content/datasets/` → set `CSV_PATH`
2. Gunakan Drive
3. Jika tidak ada, buat data sintetis untuk demo

In [2]:
CSV_PATH = "/content/datasets/contoh.csv"  # ganti jika perlu
os.makedirs('/content/datasets', exist_ok=True)

if not os.path.exists(CSV_PATH):
    print("File tidak ditemukan, membuat data sintetis untuk demo...")
    rng = np.random.default_rng(42)
    n = 500
    df = pd.DataFrame({
        'order_id': np.arange(1, n+1),
        'shift': rng.choice(['pagi','siang','malam'], size=n),
        'machine_speed': rng.normal(100, 15, size=n),
        'temperature_c': rng.normal(32, 3, size=n),
        'downtime_min': np.abs(rng.normal(10, 8, size=n)),
        'is_defect': rng.choice([0,1], size=n, p=[0.85,0.15])
    })
    # Missing & outlier
    df.loc[rng.choice(df.index, 20, replace=False), 'machine_speed'] = np.nan
    df.loc[rng.choice(df.index, 10, replace=False), 'shift'] = np.nan
    df.loc[rng.choice(df.index, 5, replace=False), 'temperature_c'] = df['temperature_c'].mean() + 6*df['temperature_c'].std()
else:
    df = pd.read_csv(CSV_PATH)

df.head()

## 2) Cast tipe & ringkas missing

In [3]:
# Cast tipe ringkas
if 'shift' in df.columns:
    df['shift'] = df['shift'].astype('category')

missing_before = df.isna().mean().sort_values(ascending=False)
print("Missing ratio BEFORE:\n", missing_before)

## 3) Dedup by-features (opsional)
Anggap baris duplikat jika **semua fitur selain `order_id` sama**.

In [4]:
feature_cols = [c for c in df.columns if c.lower() != 'order_id']

# Stabilkan numerik untuk dedup (opsional)
df_tmp = df.copy()
for c in feature_cols:
    if pd.api.types.is_numeric_dtype(df_tmp[c]):
        df_tmp[c] = df_tmp[c].round(6)

rows_before = len(df_tmp)
df_tmp = df_tmp.drop_duplicates(subset=feature_cols, keep='first')
print("Dedup by-features — dropped:", rows_before - len(df_tmp))

# lanjutkan dengan df_dedup sebagai basis imputasi sederhana
df_dedup = df_tmp.copy()

## 4) Imputasi sederhana pada kolom target
- `shift` → modus (kalau semua NaN, fallback `'Unknown'`)
- `machine_speed`, `temperature_c`, `downtime_min` → median

Ini menghasilkan **`df_clean_min.csv`** (hasil minimal yang bisa dipakai untuk EDA struktural).

In [5]:
df_clean_min = df_dedup.copy()

# Imputasi shift (kategorik)
if 'shift' in df_clean_min.columns:
    mode_val = df_clean_min['shift'].mode(dropna=True)
    fill_val = mode_val.iloc[0] if not mode_val.empty else 'Unknown'
    df_clean_min['shift'] = df_clean_min['shift'].cat.add_categories(['Unknown']) if hasattr(df_clean_min['shift'], 'cat') else df_clean_min['shift']
    df_clean_min['shift'] = df_clean_min['shift'].fillna(fill_val)

# Imputasi numerik (median)
for col in ['machine_speed', 'temperature_c', 'downtime_min']:
    if col in df_clean_min.columns:
        median_val = df_clean_min[col].median(skipna=True)
        df_clean_min[col] = pd.to_numeric(df_clean_min[col], errors='coerce')
        df_clean_min[col] = df_clean_min[col].fillna(median_val)

missing_after_min = df_clean_min.isna().mean().sort_values(ascending=False)
print("Missing ratio AFTER (df_clean_min):\n", missing_after_min)

os.makedirs('/content/data/interim', exist_ok=True)
min_path = '/content/data/interim/df_clean_min.csv'
df_clean_min.to_csv(min_path, index=False)
min_path

## 5) Pipeline siap-model (tanpa kebocoran)
Split train/test dulu, lalu lakukan **imputasi + encoding + scaling** di pipeline.

In [6]:
target = 'is_defect' if 'is_defect' in df.columns else None
if target is None:
    raise ValueError("Kolom target 'is_defect' tidak ditemukan. Silakan set target sesuai dataset Anda.")

X = df_dedup.drop(columns=[target])
y = df_dedup[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

num_features = [c for c in X_train.select_dtypes(include=['number']).columns if c != target]
cat_features = X_train.select_dtypes(include=['object','category','bool']).columns.tolist()

from sklearn import set_config
set_config(transform_output="pandas")

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, num_features),
        ('cat', categorical_pipeline, cat_features)
    ]
)
clf = Pipeline(steps=[('preprocess', preprocess), ('model', LogisticRegression(max_iter=1000))])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Materialisasi dataset siap-model
X_train_ready = clf.named_steps['preprocess'].fit_transform(X_train)
X_test_ready  = clf.named_steps['preprocess'].transform(X_test)

train_ready = X_train_ready.copy(); train_ready[target] = y_train.to_numpy()
test_ready  = X_test_ready.copy();  test_ready[target]  = y_test.to_numpy()

os.makedirs('/content/data/processed', exist_ok=True)
train_path = '/content/data/processed/train_ready.csv'
test_path  = '/content/data/processed/test_ready.csv'
train_ready.to_csv(train_path, index=False)
test_ready.to_csv(test_path, index=False)
train_path, test_path

## 6) Ringkasan
- `df_clean_min.csv` (interim): dedup by-features + imputasi sederhana pada kolom target
- `train_ready.csv` & `test_ready.csv` (processed): **siap-model** via pipeline (imputasi + One-Hot + scaling) tanpa kebocoran
- Semua perubahan bisa dicatat di **cleaning_log** dan **data_dictionary**